# משימת סיווג טקסט

במודול זה נתחיל עם משימת סיווג טקסט פשוטה המבוססת על מערך הנתונים **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)**: נסווג כותרות חדשות לאחת מתוך 4 קטגוריות: עולם, ספורט, עסקים ומדע/טכנולוגיה.

## מערך הנתונים

כדי לטעון את מערך הנתונים, נשתמש ב-API של **[TensorFlow Datasets](https://www.tensorflow.org/datasets)**.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

כעת אנו יכולים לגשת לחלקי האימון והמבחן של מערך הנתונים באמצעות `dataset['train']` ו-`dataset['test']` בהתאמה:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


בואו נדפיס את 10 הכותרות החדשות הראשונות מהמאגר שלנו:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## וקטוריזציה של טקסט

עכשיו אנחנו צריכים להמיר טקסט ל**מספרים** שניתן לייצג כטנסורים. אם אנחנו רוצים ייצוג ברמת המילים, עלינו לבצע שני דברים:

* להשתמש ב**טוקנייזר** כדי לפצל את הטקסט ל**טוקנים**.
* לבנות **אוצר מילים** של אותם טוקנים.

### הגבלת גודל אוצר המילים

בדוגמה של מערך הנתונים AG News, גודל אוצר המילים די גדול, יותר מ-100 אלף מילים. באופן כללי, אנחנו לא צריכים מילים שמופיעות לעיתים רחוקות בטקסט — רק כמה משפטים יכילו אותן, והמודל לא ילמד מהן. לכן, יש היגיון להגביל את גודל אוצר המילים למספר קטן יותר על ידי העברת פרמטר לבנאי של הוקטורייזר:

שני השלבים הללו יכולים להתבצע באמצעות שכבת **TextVectorization**. בואו ניצור את אובייקט הוקטורייזר, ואז נקרא למתודה `adapt` כדי לעבור על כל הטקסט ולבנות אוצר מילים:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **שימו לב** שאנחנו משתמשים רק בתת-קבוצה של כל מערך הנתונים כדי לבנות אוצר מילים. אנחנו עושים זאת כדי להאיץ את זמן הביצוע ולא להשאיר אתכם ממתינים. עם זאת, אנחנו לוקחים את הסיכון שחלק מהמילים מכלל מערך הנתונים לא ייכללו באוצר המילים, ויתעלמו מהן במהלך האימון. לכן, שימוש בגודל אוצר המילים המלא והרצה על כל מערך הנתונים במהלך `adapt` עשויים להעלות את הדיוק הסופי, אך לא באופן משמעותי.

כעת נוכל לגשת לאוצר המילים בפועל:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


באמצעות הוקטורייזר, אנו יכולים בקלות לקודד כל טקסט למערך של מספרים:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## ייצוג טקסט בשיטת Bag-of-words

מכיוון שמילים מייצגות משמעות, לפעמים ניתן להבין את המשמעות של טקסט רק על ידי התבוננות במילים הבודדות, בלי קשר לסדר שלהן במשפט. לדוגמה, כאשר מסווגים חדשות, מילים כמו *מזג אוויר* ו-*שלג* עשויות להצביע על *תחזית מזג אוויר*, בעוד שמילים כמו *מניות* ו-*דולר* עשויות להצביע על *חדשות כלכליות*.

ייצוג וקטורי בשיטת **Bag-of-words** (BoW) הוא הייצוג הווקטורי המסורתי הפשוט ביותר להבנה. כל מילה מקושרת לאינדקס בווקטור, ואלמנט בווקטור מכיל את מספר הפעמים שהמילה מופיעה במסמך נתון.

![תמונה שמציגה כיצד ייצוג וקטורי בשיטת Bag-of-words מיוצג בזיכרון.](../../../../../translated_images/he/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: ניתן גם לחשוב על BoW כסכום של כל הווקטורים המקודדים בשיטת one-hot עבור המילים הבודדות בטקסט.

להלן דוגמה ליצירת ייצוג בשיטת Bag-of-words באמצעות ספריית הפייתון Scikit Learn:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

אנחנו יכולים גם להשתמש בווקטורייזר של Keras שהגדרנו למעלה, להמיר כל מספר של מילה לקידוד one-hot ולהוסיף את כל הווקטורים הללו יחד:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **הערה**: ייתכן שתופתעו שהתוצאה שונה מהדוגמה הקודמת. הסיבה לכך היא שבדוגמה של Keras, אורך הווקטור תואם לגודל אוצר המילים, שנבנה מכלל מאגר הנתונים של AG News, בעוד שבדוגמה של Scikit Learn בנינו את אוצר המילים מתוך טקסט הדוגמה באופן מיידי.


## אימון מסווג BoW

עכשיו, לאחר שלמדנו כיצד לבנות את ייצוג תיקיית המילים (bag-of-words) של הטקסט שלנו, בואו נאמן מסווג שמשתמש בו. ראשית, עלינו להמיר את מערך הנתונים שלנו לייצוג תיקיית המילים. ניתן לעשות זאת באמצעות הפונקציה `map` באופן הבא:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

עכשיו נגדיר רשת עצבית מסווגת פשוטה שמכילה שכבה ליניארית אחת. גודל הקלט הוא `vocab_size`, וגודל הפלט מתאים למספר הקטגוריות (4). מכיוון שאנחנו פותרים משימת סיווג, פונקציית ההפעלה הסופית היא **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


מכיוון שיש לנו 4 קטגוריות, דיוק של מעל 80% נחשב לתוצאה טובה.

## אימון מסווג כרשת אחת

מכיוון שהוקטורייזר הוא גם שכבת Keras, אנחנו יכולים להגדיר רשת שכוללת אותו ולאמן אותה מקצה לקצה. בצורה הזו, אין צורך לוקטור את מערך הנתונים באמצעות `map`, ואפשר פשוט להעביר את מערך הנתונים המקורי כקלט לרשת.

> **הערה**: עדיין נצטרך להחיל פעולות `map` על מערך הנתונים כדי להמיר שדות ממילונים (כמו `title`, `description` ו-`label`) לטאפלים. עם זאת, כאשר טוענים נתונים מהדיסק, ניתן לבנות מראש מערך נתונים עם המבנה הנדרש.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## ביגרמים, טריגרמים ו-n-גרמים

אחת המגבלות של גישת תיקיית המילים היא שחלק מהמילים הן חלק מביטויים רב-מילתיים. לדוגמה, המילה 'נקניקייה' ('hot dog') בעלת משמעות שונה לחלוטין מהמילים 'חם' ('hot') ו'כלב' ('dog') בהקשרים אחרים. אם נייצג תמיד את המילים 'חם' ו'כלב' באמצעות אותם וקטורים, זה עלול לבלבל את המודל שלנו.

כדי להתמודד עם זה, **ייצוגי n-גרמים** משמשים לעיתים קרובות בשיטות לסיווג מסמכים, שבהן התדירות של כל מילה, זוג מילים או שלישיית מילים היא תכונה שימושית לאימון מסווגים. בייצוגי ביגרמים, לדוגמה, נוסיף את כל זוגות המילים לאוצר המילים, בנוסף למילים המקוריות.

להלן דוגמה כיצד ליצור ייצוג תיקיית מילים של ביגרמים באמצעות Scikit Learn:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

החיסרון העיקרי של גישת ה-n-gram הוא שהגודל של אוצר המילים מתחיל לגדול בקצב מהיר מאוד. בפועל, עלינו לשלב את הייצוג של n-gram עם טכניקת צמצום ממדים, כמו *embeddings*, עליה נדון ביחידה הבאה.

כדי להשתמש בייצוג n-gram במאגר הנתונים **AG News**, עלינו להעביר את הפרמטר `ngrams` לבנאי של `TextVectorization`. האורך של אוצר מילים של ביגרם הוא **גדול משמעותית**, ובמקרה שלנו הוא יותר מ-1.3 מיליון טוקנים! לכן, יש היגיון להגביל גם את טוקני הביגרם למספר סביר כלשהו.

נוכל להשתמש באותו קוד כמו קודם כדי לאמן את הסיווג, אך זה יהיה מאוד לא יעיל מבחינת זיכרון. ביחידה הבאה, נאמן את סיווג הביגרם באמצעות embeddings. בינתיים, תוכלו להתנסות באימון סיווג הביגרם במחברת זו ולראות אם תוכלו להשיג דיוק גבוה יותר.


## חישוב אוטומטי של וקטורי BoW

בדוגמה למעלה חישבנו וקטורי BoW באופן ידני על ידי סכימת הקידודים הבינאריים של מילים בודדות. עם זאת, הגרסה האחרונה של TensorFlow מאפשרת לנו לחשב וקטורי BoW באופן אוטומטי על ידי העברת הפרמטר `output_mode='count` לקונסטרקטור של הווקטורייזר. זה הופך את ההגדרה והאימון של המודל שלנו לקלים משמעותית:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## תדירות מונחים - תדירות מסמכים הפוכה (TF-IDF)

בייצוג BoW, משקל ההופעות של מילים מחושב באותה שיטה ללא קשר למילה עצמה. עם זאת, ברור שמילים נפוצות כמו *a* ו-*in* הן הרבה פחות חשובות לסיווג מאשר מונחים ייחודיים. ברוב משימות עיבוד שפה טבעית (NLP), יש מילים שהן רלוונטיות יותר מאחרות.

**TF-IDF** הוא קיצור של **תדירות מונחים - תדירות מסמכים הפוכה**. זהו וריאנט של ייצוג תיק-מילים (bag-of-words), שבו במקום ערך בינארי 0/1 שמציין את הופעת המילה במסמך, נעשה שימוש בערך עשרוני שמבוסס על תדירות הופעת המילה בקורפוס.

באופן פורמלי יותר, המשקל $w_{ij}$ של מילה $i$ במסמך $j$ מוגדר כך:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
כאשר:
* $tf_{ij}$ הוא מספר ההופעות של $i$ ב-$j$, כלומר ערך BoW שראינו קודם
* $N$ הוא מספר המסמכים באוסף
* $df_i$ הוא מספר המסמכים שמכילים את המילה $i$ בכל האוסף

ערך TF-IDF $w_{ij}$ גדל באופן יחסי למספר הפעמים שמילה מופיעה במסמך, ומותאם לפי מספר המסמכים בקורפוס שמכילים את המילה. זה עוזר לפצות על העובדה שיש מילים שמופיעות בתדירות גבוהה יותר מאחרות. לדוגמה, אם מילה מופיעה *בכל* המסמכים באוסף, אז $df_i=N$, ו-$w_{ij}=0$, והמונחים הללו יוזנחו לחלוטין.

ניתן ליצור בקלות וקטוריזציה של TF-IDF לטקסט באמצעות Scikit Learn:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

בקראס, ניתן לחשב באופן אוטומטי את תדירויות TF-IDF באמצעות העברת הפרמטר `output_mode='tf-idf'` לשכבת `TextVectorization`. בואו נחזור על הקוד שהשתמשנו בו קודם כדי לראות אם שימוש ב-TF-IDF מעלה את הדיוק:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## מסקנה

למרות שייצוגי TF-IDF מספקים משקלי תדירות למילים שונות, הם אינם מסוגלים לייצג משמעות או סדר. כפי שאמר הבלשן המפורסם ג'יי. אר. פירת' בשנת 1935: "המשמעות המלאה של מילה תמיד תלויה בהקשר, ואין לקחת ברצינות שום מחקר על משמעות ללא הקשר." בהמשך הקורס נלמד כיצד ללכוד מידע הקשרי מטקסט באמצעות מודלים לשוניים.



---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור סמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי אדם. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
